## Rumusan Masalah
- Menggunakan Closenes Centrality untuk mencari kedekatan antar user yang memiliki ketertarikan sama mengenai vaksin, dan diasumsikan dalam kedekatan tersebut sekolompok user memiliki ego atau opini yang sama mengenai vaksin
- Mencari user yang reply tweet dari @infoJakarta, dan dilakukan penyelidikan lanjut kepada user terbut dengan mencari kata kunci yang sama, dan melihat apakah terdapat user yang sama yang terhubung dengan @infoJakarta
- Selain itu juga mengidentifikasi ego dari kelompok yang didapat, apakah mendukunga atau mengkritik

In [ ]:
import sys
!{sys.executable} -m pip install twint
!{sys.executable} -m pip install nest_asyncio
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install networkx

In [1]:
import twint
import nest_asyncio
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

nest_asyncio.apply()

## Before Vaccine

In [3]:
c = twint.Config()
c.Username = 'infoJakarta'
c.Since = "2020-08-01" #batas awal tanggal pengambilan tweet
c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
for key in ['vaksin', 'covid']:
    c.Search = key
    c.Output = './data/before.json' # save to json
    twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 8 Tweets from @infoJakarta.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 11 Tweets from @infoJakarta.


## After Vaccine

In [4]:
c = twint.Config()
c.Username = 'infoJakarta'
c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
for key in ['vaksin', 'covid']:
    c.Search = key
    c.Output = './data/after.json' # save to json
    twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 8 Tweets from @infoJakarta.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 11 Tweets from @infoJakarta.


In [2]:
df_before = pd.read_json('./data/before.json' , lines = True)
df_after = pd.read_json('./data/after.json' , lines = True)

NameError: name 'pd' is not defined

In [1]:
df_before = df_before[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

df_before

NameError: name 'df_before' is not defined

In [ ]:
import sys
!{sys.executable} -m pip install selenium

In [ ]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import numpy as np
from datetime import datetime, timedelta

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
# chrome_options.add_argument('headless') # untuk menutup browser

#specify the path to chromedriver.exe
driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

In [ ]:
driver.get('https://www.twitter.com/login')
sleep(2)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('dyingrabbit_') # Ubah username sesuai akun pribadi

my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

In [ ]:
data_tweets = df[['tweet','mentions','replies_count','retweets_count','likes_count','hashtags','reply_to','link']]

In [ ]:
def get_list_likes(link):
    list_likes = []
    driver.get(link + '/likes')#open link
    sleep(5) #jeda waktu
    
    #scroll to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        likes_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Liked by"]') #element dasar likes
        if likes_container: #jika ada maka
            likes = likes_container.find_elements_by_xpath('//div[@dir="ltr"]/span[@class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"]')
            for like in likes:
                list_likes.append(like.text)
    except:
        print(list_likes)
        return list_likes
    print(list_likes)
    return list_likes



def get_list_retweets(link):
    list_retweets = []
    driver.get(link + '/retweets') #open link
    sleep(5)#jeda waktu
    
    #scroll to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        retweets_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Retweeted by"]')
        if retweets_container:
            retweets = retweets_container.find_elements_by_xpath('//div[@dir="ltr"]/span[@class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"]')
            for retweet in retweets:
                list_retweets.append(retweet.text)
    except:
        pass
    print(list_retweets)
    return list_retweets

def get_list_replies(link):
    list_replies = []
    driver.get(link)
    sleep(5)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        replies_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Conversation"]')
        if replies_container:
            replies = replies_container.find_elements_by_xpath('//div[@dir="ltr"]/span[@class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"]')
            for reply in replies:
                list_replies.append(reply.text)
    except:
        print(list_replies)
        return list_replies
    print(list_replies)
    return list_replies

In [ ]:
arr_temp = []
for i, row in data_tweets.iterrows():
     arr_temp.append(get_list_likes(row['link']))   

data_tweets['list_likes'] = arr_temp
data_tweets['list_likes']

In [ ]:
arr_temp = []
for i, row in data_tweets.iterrows():
     arr_temp.append(get_list_retweets(row['link']))

data_tweets['list_retweets'] = arr_temp
data_tweets['list_retweets']

In [ ]:
arr_temp = []
for i, row in data_tweets.iterrows():
     arr_temp.append(get_list_replies(row['link']))

data_tweets['list_replies'] = arr_temp
data_tweets['list_replies']

In [ ]:
 data_tweets.to_json('data/Scraping.json')

In [ ]:
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install scipy

In [ ]:
twt_df = pd.read_json('data/Scraping.json')
twt_df.head()

In [ ]:
arr_tmp = []
for i,row in twt_df.iterrows():
    liker = row['list_likes']
    idx = -1
    for like in liker:
        idx+=1
        if(like==''):
            break
    arr_tmp.append(liker[:idx])
    
arr_tmp
twt_df['list_likes'] = arr_tmp
twt_df

In [ ]:
arr_tmp = []
for i,row in twt_df.iterrows():
    rts = row['list_retweets']
    idx = -1
    for rt in rts:
        idx+=1
        if(rt==''):
            break
    arr_tmp.append(rts[:idx])
    
arr_tmp
twt_df['list_retweets'] = arr_tmp
twt_df

In [ ]:
arr_tmp = []
for i,row in twt_df.iterrows():
    reps = row['list_replies']
    if '@infojakarta' in reps:
        reps.remove('@infojakarta')
    arr_tmp.append(reps)
    
twt_df['list_replies'] = arr_tmp
twt_df

In [ ]:
 twt_df.to_json('data/Scraping-clean.json')

In [ ]:
twt_df = pd.read_json('data/Scraping-clean.json')

In [ ]:
def draw(G, pos, measures, measure_name, graph_name):
    plt.figure(figsize=(20,20))
    ax = plt.gca()
    nodes = nx.draw_networkx_nodes(G, pos, node_size=250, cmap=plt.cm.plasma, 
                                   node_color=list(measures.values()),
                                   nodelist=measures.keys(), ax=ax)
    nodes.set_norm(mcolors.SymLogNorm(linthresh=0.01, linscale=1, base=10))
#     labels = nx.draw_networkx_labels(G, pos)
    edges = nx.draw_networkx_edges(G, pos, ax=ax)

    plt.title(measure_name)
    plt.colorbar(nodes)
    plt.axis('off')
    plt.savefig(graph_name, format='png')
    plt.show()

In [ ]:
G = nx.Graph()
for i,row in twt_df.iterrows():
    for rep in row['list_replies']:
        G.add_edge(row['tweet'], rep)

In [ ]:
centrality = nx.degree_centrality(G)
for key in centrality:
    print(key, ' : ', centrality[key])

In [ ]:
draw(G, nx.spring_layout(G, k=0.05, iterations=20), nx.degree_centrality(G), 'Replies Degree Centrality', 'replies.png')

In [ ]:
H = nx.Graph()
for i,row in twt_df.iterrows():
    for like in row['list_likes']:
        H.add_edge(row['tweet'], like)

In [ ]:
centrality = nx.degree_centrality(H)
for key in centrality:
    print(key, ' : ', centrality[key])

In [ ]:
draw(H, nx.spring_layout(H, k=0.05, iterations=20), nx.degree_centrality(H), 'Likes Degree Centrality', 'likes.png')

In [ ]:
K = nx.Graph()
for i,row in twt_df.iterrows():
    for rt in row['list_retweets']:
        K.add_edge(row['tweet'], rt)

In [ ]:
centrality = nx.degree_centrality(K)
for key in centrality:
    print(key, ' : ', centrality[key])

In [ ]:
draw(K, nx.spring_layout(K, k=0.05, iterations=20), nx.degree_centrality(K), 'Retweets Degree Centrality','rt.png')